<a href="https://colab.research.google.com/github/ajaysaikiran2208/Recommendation-Systems/blob/main/Drug_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##***K Nearest Neighbor recommendation system*** for Drug Recommendation 

K Nearest Neighbour finds the point depending upon the distance.we have,

***1.Euclidean Distance*** 

***2.Manhattan Distance***

In the Case of **Classification** the new point will be having the value of class which has **highest points** in them.For example if we are classifying Male and Female with K=5[It should be always an odd value] and in this case 3 values are male and 2 values are female then new point will be male.

In the case of **Regression** the new point will be having the value of the **mean** of all the points since this is a continous data

***Outliers*** and ***Imbalance*** data will impact the performance of the model.

##Creating Recommendation Systems using Nearest Neighbors

Importing Libraries

In [2]:
import pandas as pd
import numpy as np

In [3]:
drugs=pd.read_csv("/content/drugsComTrain_raw.csv")
drugs.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,03-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [4]:
drugs.drop(columns=["date"])

,uniqueID,drugName,condition,review,rating,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,37
...,...,...,...,...,...,...
214944,24222,Tretinoin,Acne,"""I started the combonation of Tretinoin Cream ...",9,47
214945,145914,Qsymia,Weight Loss,"""I lost 13 pounds in 10 days. At first I was v...",10,88
214946,69008,Medroxyprogesterone,Abnormal Uterine Bleeding,"""I had my 8th child in Nov/2015.I received a ...",1,7
214947,189187,Oxycodone,Chronic Pain,"""Oxycodone relieves 85% of my pain and 99% of...",9,46


In [7]:
combine_drug_rating=drugs.dropna(axis=0,subset=["drugName"])
drug_ratingcount=(combine_drug_rating.groupby(by=['drugName'])['condition']).count().reset_index().rename(columns={'condition':'totalratingcount'})[['drugName','totalratingcount']]
drug_ratingcount.head()

,drugName,totalratingcount
0,A + D Cracked Skin Relief,1
1,A / B Otic,2
2,Abacavir / dolutegravir / lamivudine,70
3,Abacavir / lamivudine,3
4,Abacavir / lamivudine / zidovudine,1


In [8]:
drug_ratingcount=combine_drug_rating.merge(drug_ratingcount,left_on='drugName',right_on='drugName',how='left')
drug_ratingcount.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,totalratingcount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,82
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,170
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,14
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,03-Nov-15,10,290
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,599


In [9]:
pd.set_option('display.float_format',lambda x:'%.3f'%x)
print(drug_ratingcount['totalratingcount'].describe())

count   214949.000
mean       825.266
std       1099.273
min          0.000
25%        130.000
50%        433.000
75%       1065.000
max       4891.000
Name: totalratingcount, dtype: float64


In [10]:
totalratingcount_threshold=50
rating_popular_drug=drug_ratingcount.query('totalratingcount>=@totalratingcount_threshold')
rating_popular_drug.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,totalratingcount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,82
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,170
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,03-Nov-15,10,290
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,599
5,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,368


In [11]:
rating_popular_drug.shape


(188840, 8)

In [12]:
final_dataset = rating_popular_drug.pivot_table(index='drugName',columns='condition',values='rating').fillna(0)
final_dataset.head()

condition,0</span> users found this comment helpful.,100</span> users found this comment helpful.,105</span> users found this comment helpful.,10</span> users found this comment helpful.,11</span> users found this comment helpful.,123</span> users found this comment helpful.,12</span> users found this comment helpful.,135</span> users found this comment helpful.,13</span> users found this comment helpful.,142</span> users found this comment helpful.,145</span> users found this comment helpful.,14</span> users found this comment helpful.,15</span> users found this comment helpful.,16</span> users found this comment helpful.,17</span> users found this comment helpful.,18</span> users found this comment helpful.,19</span> users found this comment helpful.,1</span> users found this comment helpful.,20</span> users found this comment helpful.,21</span> users found this comment helpful.,22</span> users found this comment helpful.,23</span> users found this comment helpful.,24</span> users found this comment helpful.,25</span> users found this comment helpful.,26</span> users found this comment helpful.,27</span> users found this comment helpful.,28</span> users found this comment helpful.,29</span> users found this comment helpful.,2</span> users found this comment helpful.,30</span> users found this comment helpful.,31</span> users found this comment helpful.,32</span> users found this comment helpful.,33</span> users found this comment helpful.,34</span> users found this comment helpful.,35</span> users found this comment helpful.,36</span> users found this comment helpful.,38</span> users found this comment helpful.,39</span> users found this comment helpful.,3</span> users found this comment helpful.,40</span> users found this comment helpful.,...,Upper Respiratory Tract Infection,Urinary Incontinence,Urinary Tract Infection,Urinary Tract Stones,Urticaria,Uterine Fibroids,Uveitis,"Uveitis, Posteri",Vaginal Yeast Infection,Varicella-Zoste,Ventricular Tachycardia,Vertig,Vitamin/Mineral Supplementation during Pregnancy/Lactation,Vulvodynia,Warts,Wegener's Granulomatosis,Weight Loss,Wolff-Parkinson-White Syndrome,Women (minoxidil),Women (oxybutynin),Zollinger-Ellison Syndrome,acial Wrinkles,actor IX Deficiency,ailure to Thrive,amilial Mediterranean Feve,atigue,cal Segmental Glomerulosclerosis,emale Infertility,eve,ibromyalgia,lic Acid Deficiency,llicular Lymphoma,m Pain Disorde,mance Anxiety,min),mis,moterol),t Pac with Cyclobenzaprine (cyclobenzaprine),ungal Pneumonia,zen Shoulde
drugName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abacavir / dolutegravir / lamivudine,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Abilify,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,10.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,7.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Abreva,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000

In [13]:
from scipy.sparse import csr_matrix
drug_features_matrix=csr_matrix(final_dataset.values)
from sklearn.neighbors import NearestNeighbors
model_knn=NearestNeighbors(metric="cosine",algorithm='brute')
model_knn.fit(drug_features_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [14]:
final_dataset.shape

(689, 647)

In [26]:
query_index=np.random.choice(final_dataset.shape[1])
print(query_index)
distances,indices=model_knn.kneighbors(final_dataset.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6)

339


In [27]:
final_dataset.head()

condition,0</span> users found this comment helpful.,100</span> users found this comment helpful.,105</span> users found this comment helpful.,10</span> users found this comment helpful.,11</span> users found this comment helpful.,123</span> users found this comment helpful.,12</span> users found this comment helpful.,135</span> users found this comment helpful.,13</span> users found this comment helpful.,142</span> users found this comment helpful.,145</span> users found this comment helpful.,14</span> users found this comment helpful.,15</span> users found this comment helpful.,16</span> users found this comment helpful.,17</span> users found this comment helpful.,18</span> users found this comment helpful.,19</span> users found this comment helpful.,1</span> users found this comment helpful.,20</span> users found this comment helpful.,21</span> users found this comment helpful.,22</span> users found this comment helpful.,23</span> users found this comment helpful.,24</span> users found this comment helpful.,25</span> users found this comment helpful.,26</span> users found this comment helpful.,27</span> users found this comment helpful.,28</span> users found this comment helpful.,29</span> users found this comment helpful.,2</span> users found this comment helpful.,30</span> users found this comment helpful.,31</span> users found this comment helpful.,32</span> users found this comment helpful.,33</span> users found this comment helpful.,34</span> users found this comment helpful.,35</span> users found this comment helpful.,36</span> users found this comment helpful.,38</span> users found this comment helpful.,39</span> users found this comment helpful.,3</span> users found this comment helpful.,40</span> users found this comment helpful.,...,Upper Respiratory Tract Infection,Urinary Incontinence,Urinary Tract Infection,Urinary Tract Stones,Urticaria,Uterine Fibroids,Uveitis,"Uveitis, Posteri",Vaginal Yeast Infection,Varicella-Zoste,Ventricular Tachycardia,Vertig,Vitamin/Mineral Supplementation during Pregnancy/Lactation,Vulvodynia,Warts,Wegener's Granulomatosis,Weight Loss,Wolff-Parkinson-White Syndrome,Women (minoxidil),Women (oxybutynin),Zollinger-Ellison Syndrome,acial Wrinkles,actor IX Deficiency,ailure to Thrive,amilial Mediterranean Feve,atigue,cal Segmental Glomerulosclerosis,emale Infertility,eve,ibromyalgia,lic Acid Deficiency,llicular Lymphoma,m Pain Disorde,mance Anxiety,min),mis,moterol),t Pac with Cyclobenzaprine (cyclobenzaprine),ungal Pneumonia,zen Shoulde
drugName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abacavir / dolutegravir / lamivudine,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Abilify,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,10.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,7.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Abreva,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000

In [28]:
for i in range(0,len(distances.flatten())):
  if i ==0:
    print('Recommendations for {0}:\n'.format(final_dataset.index[query_index]))
  else:
    print('{0}:{1},with distance of {2}:'.format(i,final_dataset.index[indices.flatten()[i]],distances.flatten()[i]))

Recommendations for Levothyroxine:

1:Synthroid,with distance of 0.1532354290483875:
2:Thyroid desiccated,with distance of 0.22508250455079837:
3:Armour Thyroid,with distance of 0.32480039721512755:
4:Hydrocodone,with distance of 0.6629061858008162:
5:Doxylamine,with distance of 0.6681167823010015:
